In [1]:
import math
import os
import torch
from PIL import Image
import numpy as np
from model import StyledGenerator
import tqdm
import matplotlib.pyplot as plt
from IPython.display import Video
import glob
import cv2
from natsort import natsorted # pip install natsort

C:\conda\envs\face_edit\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## 生成图片
def make_image(tensor):
    return (
        tensor.detach()
        .clamp_(min=-1, max=1)
        .add(1)
        .div_(2)
        .mul(255)
        .type(torch.uint8)
        .permute(0, 2, 3, 1)
        .to("cpu")
        .numpy()
    )


In [3]:
# 加载模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 载入模型
netG = StyledGenerator(512)
netG.load_state_dict(torch.load('./checkpoints/stylegan-1024px-new.model',map_location=device)["g_running"], strict=False)
netG.eval()
netG = netG.to(device)
step = int(math.log(1024, 2)) - 2

In [4]:
def generateMix(direction_np,latent_np,mode='z_mode',editscale=1):
    
    # 载入方向向量
    direction = np.load(direction_np+'.npy')
    # 载入人脸向量
    latent = torch.from_numpy(np.load(latent_np+'.npy')).to(device)
    # 
    if len(latent.shape) == 1:
        latent = latent.unsqueeze(0)
    
    if mode=='z_mode':
        # 基于Z混合
        img_name = './output/edit_single_z/{}.png'.format(editscale)
        # 计算Z方向偏移量
        latent = latent + torch.from_numpy((editscale * direction[0]).astype(np.float32)).to(device) ##计算Z方向偏移量
        latent.to(device) 
        # 生成的图片
        img_gen = netG([latent], step=step) 

    else:
        img_name = './output/edit_single_w/{}.png'.format(editscale)
        
        # 计算W方向偏移量
        w = torch.from_numpy((editscale * direction[0]).astype(np.float32)).to(device)
        latent.to(device) 
        #生成结果只由mean_style决定
        img_gen = netG([latent], mean_style = latent + w, step=step, style_weight=0) 
    
    # 保存图片
    img_ar = make_image(img_gen)
    
    return img_ar[0]
    # pil_img = Image.fromarray(img_ar[0])
    # pil_img.save(img_name) 

In [5]:
# 合称为视频
def animatePics(direction_np,latent_np,mode='z_mode',scale_low=-1,scale_high=0.7,step=0.05,img_size = (512,512),fps=25):
    
    if mode == 'z_mode':
        video_name = './output/edit_single_z/output.mp4'
    else:
        video_name = './output/edit_single_w/output.mp4'        
    # 删除原来的视频文件
    if os.path.exists(video_name):
        os.unlink(video_name)
    
        
    # 视频写入
    videoWriter = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'H264'), fps, img_size)
    
    scale = scale_low
    
    while True:
        scale += step
        scale = round(scale,4)
        
        if scale < scale_high:
            img = generateMix(direction_np,latent_np,mode=mode,editscale=scale)
            # BGR->RGB
            img = img[:,:,[2,1,0]]
            # resize
            img = cv2.resize(img,img_size)
            # 写入
            videoWriter.write(img)
        else:
            break
    videoWriter.release()   
    
    return Video(video_name,embed=True)

In [6]:
# Z:微笑变化
direction_np = './latent_directions/smile_z'
latent_np = './latents/z/fake3'

animatePics(direction_np,latent_np,scale_low=-1.2,scale_high=1.1,step=0.2,fps=15)

In [7]:
# Z:年龄变化
direction_np = './latent_directions/age_z'
latent_np = './latents/z/fake2'

animatePics(direction_np,latent_np,scale_low=-0.7,scale_high=0.7,step=0.05,fps=15)

In [8]:
# Z:性别变化
direction_np = './latent_directions/gender_z'
latent_np = './latents/z/fake3'

animatePics(direction_np,latent_np,scale_low=-1.2,scale_high=0.3,step=0.05,fps=15)

In [9]:
# W:微笑变化
direction_np = './latent_directions/smile_w'
latent_np = './latents/w/fake3'

animatePics(direction_np,latent_np,mode='w_mode',scale_low=-0.04,scale_high=0.05,step=0.005,fps=12)

In [10]:
# W:年龄变化
direction_np = './latent_directions/age_w'
latent_np = './latents/w/fake2'

animatePics(direction_np,latent_np,mode='w_mode',scale_low=-0.2,scale_high=0.17,step=0.01,fps=20)

In [11]:
# W:性别变化
direction_np = './latent_directions/gender_w'
latent_np = './latents/w/fake2'

animatePics(direction_np,latent_np,mode='w_mode',scale_low=-0.2,scale_high=0.2,step=0.05,fps=12)

In [12]:
# Z生成的W:微笑变化
direction_np = './latent_directions/smile_w'
latent_np = './latents/z/related_w/fake3'

animatePics(direction_np,latent_np,mode='w_mode',scale_low=-0.04,scale_high=0.05,step=0.005,fps=12)

In [13]:
# W:年龄变化
direction_np = './latent_directions/age_w'
latent_np = './latents/z/related_w/fake2'

animatePics(direction_np,latent_np,mode='w_mode',scale_low=-0.08,scale_high=0.12,step=0.01,fps=20)

In [14]:
# W:性别变化
direction_np = './latent_directions/gender_w'
latent_np = './latents/z/related_w/fake2'

animatePics(direction_np,latent_np,mode='w_mode',scale_low=-0.08,scale_high=0.12,step=0.01,fps=12)